<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#model-description" data-toc-modified-id="model-description-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>model description</a></span></li><li><span><a href="#training" data-toc-modified-id="training-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>training</a></span></li><li><span><a href="#hyperparameter-search" data-toc-modified-id="hyperparameter-search-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>hyperparameter search</a></span></li></ul></div>

In [ ]:
%run ./PASTEL_MTL_training_utils.ipynb

In [ ]:
os.environ['WANDB_NOTEBOOK_NAME'] = 'PASTEL train on prediction.ipynb'
import wandb
wandb.init(project="PASTEL all together", entity="fsu-dsc-cil", dir='/scratch/data_jz17d/wandb_tmp/')

# model description
Treat all tasks equally important. This can be improved by some dynamic weighting scheme.

# training

In [ ]:
my_training_args = MyTrainingArgs(selected_tasks=list(tasks2labels.keys()),
                                  model_name=f'PASTEL all together',
                                  base_model_name='bert-base-uncased',
                                  freeze_bert=False,
                                  use_pooler=False,
                                  num_epoch=5,
#                                   data_limit=30000,
                                 )

hg_training_args = TrainingArguments(
    output_dir=my_training_args.model_folder,   # output directory
    num_train_epochs=my_training_args.num_epoch,     # total number of training epochs
    per_device_train_batch_size=my_training_args.batch_size,  # batch size per device during training
    per_device_eval_batch_size=my_training_args.batch_size,   # batch size for evaluation
    warmup_steps=my_training_args.num_warmup_steps,    # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir=f"{my_training_args.model_folder}/logs",  # directory for storing logs
    logging_first_step = True, 
    evaluation_strategy="epoch",     # evaluate each `logging_steps`
    save_total_limit = 1,
    save_strategy = 'epoch',
#     load_best_model_at_end=True, # decide on loss
)

model = init_model(my_training_args)
freeze_model(model, my_training_args.freeze_bert)

train_dataset = MyDataset(my_training_args, 'train')
val_dataset = MyDataset(my_training_args, 'valid')

trainer = MyTrainer(
    model=model,   # the instantiated Transformers model to be trained
    args=hg_training_args,                  # training arguments, defined above
    tokenizer=model.tokenizer, 
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,          # evaluation dataset
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
trainer.train()

***** Running training *****
  Num examples = 31806
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 4970


Epoch,Training Loss,Validation Loss,Accuracy Country,F1 Country,Precision Country,Recall Country,Accuracy Politics,F1 Politics,Precision Politics,Recall Politics,Accuracy Tod,F1 Tod,Precision Tod,Recall Tod,Accuracy Age,F1 Age,Precision Age,Recall Age,Accuracy Education,F1 Education,Precision Education,Recall Education,Accuracy Ethnic,F1 Ethnic,Precision Ethnic,Recall Ethnic,Accuracy Gender,F1 Gender,Precision Gender,Recall Gender
1,7.494300,6.549165,0.975188,0.329146,0.325063,0.333333,0.487719,0.334302,0.502497,0.392146,0.452632,0.211075,0.351031,0.259815,0.439599,0.190923,0.269814,0.189890,0.422306,0.180475,0.323987,0.188106,0.832832,0.209616,0.314527,0.204395,0.740852,0.446754,0.522386,0.444391
2,6.360900,6.447563,0.975188,0.329146,0.325063,0.333333,0.518296,0.452237,0.472355,0.450433,0.450376,0.222400,0.266524,0.263261,0.447118,0.219009,0.247230,0.218847,0.429073,0.234381,0.368077,0.224532,0.831830,0.230729,0.326343,0.218785,0.747619,0.457247,0.519297,0.452898
3,5.758400,6.518891,0.975188,0.329146,0.325063,0.333333,0.525063,0.434295,0.501462,0.441295,0.434085,0.246019,0.410317,0.269680,0.462907,0.215438,0.255540,0.210451,0.427820,0.243944,0.386119,0.231354,0.832331,0.242526,0.386376,0.227955,0.744110,0.459202,0.505475,0.454516
4,4.946500,6.833501,0.976190,0.355716,0.658722,0.347079,0.520301,0.457075,0.479398,0.454112,0.432832,0.260159,0.327850,0.277280,0.460902,0.229632,0.244067,0.226276,0.422556,0.232733,0.353030,0.228047,0.830827,0.247910,0.342590,0.230801,0.736090,0.463777,0.483813,0.459443
5,4.335300,7.142960,0.975940,0.349271,0.658641,0.343643,0.515288,0.453988,0.467651,0.451694,0.428571,0.277243,0.348899,0.285239,0.451128,0.230202,0.238437,0.227603,0.413784,0.237682,0.359824,0.232810,0.827820,0.244369,0.328972,0.229723,0.732581,0.463065,0.479207,0.459130


***** Running Evaluation *****
  Num examples = 3990
  Batch size = 32
/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control 

Saving model checkpoint to /scratch/data_jz17d/result/PASTEL all together/20220920-14:49:21/checkpoint-3976
Configuration saved in /scratch/data_jz17d/result/PASTEL all together/20220920-14:49:21/checkpoint-3976/config.json
Model weights saved in /scratch/data_jz17d/result/PASTEL all together/20220920-14:49:21/checkpoint-3976/pytorch_model.bin
tokenizer config file saved in /scratch/data_jz17d/result/PASTEL all together/20220920-14:49:21/checkpoint-3976/tokenizer_config.json
Special tokens file saved in /scratch/data_jz17d/result/PASTEL all together/20220920-14:49:21/checkpoint-3976/special_tokens_map.json
Deleting older checkpoint [/scratch/data_jz17d/result/PASTEL all together/20220920-14:49:21/checkpoint-2982] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 3990
  Batch size = 32
/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set

TrainOutput(global_step=4970, training_loss=5.678825260599854, metrics={'train_runtime': 839.3308, 'train_samples_per_second': 189.472, 'train_steps_per_second': 5.921, 'total_flos': 2334434796900360.0, 'train_loss': 5.678825260599854, 'epoch': 5.0})

# hyperparameter search

In [ ]:
my_training_args = MyTrainingArgs(selected_tasks=list(tasks2labels.keys()),
                                  model_name=f'PASTEL all together',
                                  base_model_name='bert-base-uncased',
                                  freeze_bert=False,
                                  use_pooler=False,
                                  num_epoch=5,
#                                   data_limit=30000,
                                 )

hg_training_args = TrainingArguments(
    output_dir=my_training_args.model_folder,   # output directory
    num_train_epochs=my_training_args.num_epoch,     # total number of training epochs
    per_device_train_batch_size=my_training_args.batch_size,  # batch size per device during training
    per_device_eval_batch_size=my_training_args.batch_size,   # batch size for evaluation
    warmup_steps=my_training_args.num_warmup_steps,    # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir=f"{my_training_args.model_folder}/logs",  # directory for storing logs
    logging_first_step = True, 
    evaluation_strategy = "epoch",     # evaluate each `logging_steps`
    save_total_limit = 1,
    save_strategy = 'epoch',
    report_to = 'wandb',
#     load_best_model_at_end=True, # decide on loss
)

def model_init():
    model = init_model(my_training_args)
    freeze_model(model, my_training_args.freeze_bert)
    return model


train_dataset = MyDataset(my_training_args, 'train')
val_dataset = MyDataset(my_training_args, 'valid')

trainer = MyTrainer(
    model=model,   # the instantiated Transformers model to be trained
    args=hg_training_args,                  # training arguments, defined above
    tokenizer=model.tokenizer, 
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,          # evaluation dataset
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
    model_init=model_init,
)

PyTorch: setting up devices
loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/jz17d/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file https://huggingface.co/bert-base-uncased/resolve/main/voc

In [ ]:
from ray.tune.search.hyperopt import HyperOptSearch
from ray.tune.schedulers import ASHAScheduler

In [ ]:
best_trial = trainer.hyperparameter_search(
    direction="minimize",
    backend="ray",
    # Choose among many libraries:
    # https://docs.ray.io/en/latest/tune/api_docs/suggestion.html
    search_alg=HyperOptSearch(metric="val_loss", mode="min"),
    # Choose among schedulers:
    # https://docs.ray.io/en/latest/tune/api_docs/schedulers.html
    scheduler=ASHAScheduler(metric="val_loss", mode="min"))

No `resources_per_trial` arg was passed into `hyperparameter_search`. Setting it to a default value of 1 CPU and 1 GPU for each trial.


== Status ==
Current time: 2022-09-20 16:27:27 (running for 00:00:00.20)
Memory usage on this node: 7.7/15.1 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 64.000: None | Iter 16.000: None | Iter 4.000: None | Iter 1.000: None
Resources requested: 1.0/6 CPUs, 1.0/1 GPUs, 0.0/5.52 GiB heap, 0.0/2.76 GiB objects (0.0/1.0 accelerator_type:G)
Result logdir: /home/jz17d/ray_results/_objective_2022-09-20_16-27-27
Number of trials: 1/20 (1 RUNNING)
+---------------------+----------+---------------------+-----------------+--------------------+------------------------+---------+
| Trial name          | status   | loc                 |   learning_rate |   num_train_epochs |   per_device_train_... |    seed |
|---------------------+----------+---------------------+-----------------+--------------------+------------------------+---------|
| _objective_a4d95c13 | RUNNING  | 144.174.6.90:202458 |     1.62489e-06 |                  4 |                     64 | 34.8974 |
+---------------------+

(_objective pid=202458) Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
(_objective pid=202458) - This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
(_objective pid=202458) - This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
(_objective pid=202458) wandb: Currently logged in as: cpuyyp (fsu-dsc-cil). Use `w

== Status ==
Current time: 2022-09-20 16:27:52 (running for 00:00:25.16)
Memory usage on this node: 10.3/15.1 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 64.000: None | Iter 16.000: None | Iter 4.000: None | Iter 1.000: None
Resources requested: 1.0/6 CPUs, 1.0/1 GPUs, 0.0/5.52 GiB heap, 0.0/2.76 GiB objects (0.0/1.0 accelerator_type:G)
Result logdir: /home/jz17d/ray_results/_objective_2022-09-20_16-27-27
Number of trials: 2/20 (1 PENDING, 1 RUNNING)
+---------------------+----------+---------------------+-----------------+--------------------+------------------------+---------+
| Trial name          | status   | loc                 |   learning_rate |   num_train_epochs |   per_device_train_... |    seed |
|---------------------+----------+---------------------+-----------------+--------------------+------------------------+---------|
| _objective_a4d95c13 | RUNNING  | 144.174.6.90:202458 |     1.62489e-06 |                  4 |                     64 | 34.8974 |
| _objectiv

(_objective pid=202458) wandb: Tracking run with wandb version 0.13.3
(_objective pid=202458) wandb: Run data is saved locally in /home/jz17d/ray_results/_objective_2022-09-20_16-27-27/_objective_a4d95c13_1_learning_rate=0.0000,num_train_epochs=4,per_device_train_batch_size=64,seed=34.8974_2022-09-20_16-27-27/wandb/run-20220920_162750-2a4c5vhy
(_objective pid=202458) wandb: Run `wandb offline` to turn off syncing.
(_objective pid=202458) wandb: Syncing run /scratch/data_jz17d/result/PASTEL all together/20220920-16:27:08
(_objective pid=202458) wandb: ⭐️ View project at https://wandb.ai/fsu-dsc-cil/huggingface
(_objective pid=202458) wandb: 🚀 View run at https://wandb.ai/fsu-dsc-cil/huggingface/runs/2a4c5vhy
  0%|          | 0/1988 [00:00<?, ?it/s]
(_objective pid=202458) 2022-09-20 16:27:55,499	ERROR function_trainable.py:298 -- Runner Thread raised error.
(_objective pid=202458) Traceback (most recent call last):
(_objective pid=202458)   File "/home/jz17d/anaconda3/envs/torch/lib/pyt

Result for _objective_a4d95c13:
  date: 2022-09-20_16-27-37
  experiment_id: 31364334ba804057bb7b77a02cc54772
  hostname: hallway-a.sc.local
  node_ip: 144.174.6.90
  pid: 202458
  timestamp: 1663705657
  trial_id: a4d95c13
  
== Status ==
Current time: 2022-09-20 16:28:00 (running for 00:00:33.22)
Memory usage on this node: 7.9/15.1 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 64.000: None | Iter 16.000: None | Iter 4.000: None | Iter 1.000: None
Resources requested: 1.0/6 CPUs, 1.0/1 GPUs, 0.0/5.52 GiB heap, 0.0/2.76 GiB objects (0.0/1.0 accelerator_type:G)
Result logdir: /home/jz17d/ray_results/_objective_2022-09-20_16-27-27
Number of trials: 3/20 (1 ERROR, 1 PENDING, 1 RUNNING)
+---------------------+----------+---------------------+-----------------+--------------------+------------------------+---------+
| Trial name          | status   | loc                 |   learning_rate |   num_train_epochs |   per_device_train_... |    seed |
|---------------------+----------+----

2022-09-20 16:28:04,803	WARNING tune.py:686 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 


== Status ==
Current time: 2022-09-20 16:28:05 (running for 00:00:38.31)
Memory usage on this node: 9.9/15.1 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 64.000: None | Iter 16.000: None | Iter 4.000: None | Iter 1.000: None
Resources requested: 1.0/6 CPUs, 1.0/1 GPUs, 0.0/5.52 GiB heap, 0.0/2.76 GiB objects (0.0/1.0 accelerator_type:G)
Result logdir: /home/jz17d/ray_results/_objective_2022-09-20_16-27-27
Number of trials: 3/20 (1 ERROR, 1 PENDING, 1 RUNNING)
+---------------------+----------+---------------------+-----------------+--------------------+------------------------+---------+
| Trial name          | status   | loc                 |   learning_rate |   num_train_epochs |   per_device_train_... |    seed |
|---------------------+----------+---------------------+-----------------+--------------------+------------------------+---------|
| _objective_aae03652 | RUNNING  | 144.174.6.90:202563 |     9.45277e-06 |                  4 |                      4 | 13.5222 |
| _

KeyboardInterrupt: 

In [ ]:
wandb.finish() 

eval/accuracy_age,▁▄█▇█
eval/accuracy_country,▅█▅▁█
eval/accuracy_education,▁██▄▅
eval/accuracy_ethnic,▆▇█▁▁
eval/accuracy_gender,▃▆█▃▁
eval/accuracy_politics,▁▁█▇▃
eval/accuracy_tod,█▆▄▄▁
eval/f1_age,▁▇▇▇█
eval/f1_country,▁▅▁▁█
eval/f1_education,▁▆███
eval/f1_ethnic,▁▅▅██


wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
--- Logging error ---
Traceback (most recent call last):
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/logging/__init__.py", line 1087, in emit
    self.flush()
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/logging/__init__.py", line 1067, in flush
    self.stream.flush()
OSError: [Errno 116] Stale file handle
Call stack:
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/threading.py", line 930, in _bootstrap
    self._bootstrap_inner()
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/wandb/sdk/internal/file_stream.py", line 489, in _thread_except_body
    self._thread_body()
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.

wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
--- Logging error ---
Traceback (most recent call last):
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/logging/__init__.py", line 1087, in emit
    self.flush()
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/logging/__init__.py", line 1067, in flush
    self.stream.flush()
OSError: [Errno 116] Stale file handle
Call stack:
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/threading.py", line 930, in _bootstrap
    self._bootstrap_inner()
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/wandb/sdk/internal/file_stream.py", line 489, in _thread_except_body
    self._thread_body()
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.

wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
--- Logging error ---
Traceback (most recent call last):
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/logging/__init__.py", line 1087, in emit
    self.flush()
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/logging/__init__.py", line 1067, in flush
    self.stream.flush()
OSError: [Errno 116] Stale file handle
Call stack:
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/threading.py", line 930, in _bootstrap
    self._bootstrap_inner()
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/wandb/sdk/internal/file_stream.py", line 489, in _thread_except_body
    self._thread_body()
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.

wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
--- Logging error ---
Traceback (most recent call last):
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/logging/__init__.py", line 1087, in emit
    self.flush()
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/logging/__init__.py", line 1067, in flush
    self.stream.flush()
OSError: [Errno 116] Stale file handle
Call stack:
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/threading.py", line 930, in _bootstrap
    self._bootstrap_inner()
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/wandb/sdk/internal/file_stream.py", line 489, in _thread_except_body
    self._thread_body()
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.

...

wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
--- Logging error ---
Traceback (most recent call last):
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/logging/__init__.py", line 1087, in emit
    self.flush()
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/logging/__init__.py", line 1067, in flush
    self.stream.flush()
OSError: [Errno 116] Stale file handle
Call stack:
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/threading.py", line 930, in _bootstrap
    self._bootstrap_inner()
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/wandb/sdk/internal/file_stream.py", line 489, in _thread_except_body
    self._thread_body()
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.

wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
--- Logging error ---
Traceback (most recent call last):
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/logging/__init__.py", line 1087, in emit
    self.flush()
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/logging/__init__.py", line 1067, in flush
    self.stream.flush()
OSError: [Errno 116] Stale file handle
Call stack:
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/threading.py", line 930, in _bootstrap
    self._bootstrap_inner()
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/wandb/sdk/internal/file_stream.py", line 489, in _thread_except_body
    self._thread_body()
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.

wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
--- Logging error ---
Traceback (most recent call last):
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/logging/__init__.py", line 1087, in emit
    self.flush()
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/logging/__init__.py", line 1067, in flush
    self.stream.flush()
OSError: [Errno 116] Stale file handle
Call stack:
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/threading.py", line 930, in _bootstrap
    self._bootstrap_inner()
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/wandb/sdk/internal/file_stream.py", line 489, in _thread_except_body
    self._thread_body()
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.

wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
--- Logging error ---
Traceback (most recent call last):
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/logging/__init__.py", line 1087, in emit
    self.flush()
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/logging/__init__.py", line 1067, in flush
    self.stream.flush()
OSError: [Errno 116] Stale file handle
Call stack:
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/threading.py", line 930, in _bootstrap
    self._bootstrap_inner()
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/wandb/sdk/internal/file_stream.py", line 489, in _thread_except_body
    self._thread_body()
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.

wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
--- Logging error ---
Traceback (most recent call last):
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/logging/__init__.py", line 1087, in emit
    self.flush()
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/logging/__init__.py", line 1067, in flush
    self.stream.flush()
OSError: [Errno 116] Stale file handle
Call stack:
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/threading.py", line 930, in _bootstrap
    self._bootstrap_inner()
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/wandb/sdk/internal/file_stream.py", line 489, in _thread_except_body
    self._thread_body()
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.